<h1> Финальный проект <span class="tocSkip"></span></h1>

# Инициализирующий режим
## Парсинг сайтов новостей

In [2]:
import requests as req 
from bs4 import BeautifulSoup   
import time
import json
import tqdm

import numpy as np
import pandas as pd

In [3]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [4]:
sites = { 'Фонтонка.ру' : ['https://www.fontanka.ru/fontanka.rss', ''],
                  'LENTA.RU' : ['https://lenta.ru/rss', ''],
                  'TASS' : ['https://tass.ru/rss/v2.xml', ''],
                  'ВЕДОМОСТИ' : ['https://www.vedomosti.ru/rss/news', '']
           }

In [5]:
sites

{'Фонтонка.ру': ['https://www.fontanka.ru/fontanka.rss', ''],
 'LENTA.RU': ['https://lenta.ru/rss', ''],
 'TASS': ['https://tass.ru/rss/v2.xml', ''],
 'ВЕДОМОСТИ': ['https://www.vedomosti.ru/rss/news', '']}

In [6]:
sites 

{'Фонтонка.ру': ['https://www.fontanka.ru/fontanka.rss', ''],
 'LENTA.RU': ['https://lenta.ru/rss', ''],
 'TASS': ['https://tass.ru/rss/v2.xml', ''],
 'ВЕДОМОСТИ': ['https://www.vedomosti.ru/rss/news', '']}

In [7]:
# создадим пустые DataFrames, куда будем записывать данные
data = {"data" : []  }
data_dirty = {"data" : []  }
df = pd.DataFrame(data['data'])
df_dirty = pd.DataFrame(data_dirty['data'])

In [8]:
# Функция парсит данные с сайтов из спика sites, записывает "грязные данные" в json файл,
# а также возвращает два DataFrame: с только необходимыми нам данными и со всеми спарсенными данными

def parsing(df, df_dirty):        
        
    for site in range(len(sites)):
        
        exitFlag=False
        
        data = {"data" : [] 
               }
        data_dirty = {"data" : []  }
       
        url = list(sites.values())[site][0]
                
        resp = req.get(url)
        soup  = BeautifulSoup(resp.content, features="lxml")

        tag_item = soup.find_all('item')
        
        for tag in tag_item:
            
            title = tag.find('title')
            
            if title.text != list(sites.values())[site][1]: 
                exitFlag=True 
                
                category = tag.find('category')

                date = tag.find('pubdate')

                data['data'].append({"title" : title.text, "category": category.text,
                                     "pubdate" : date.text, "site" : list(sites.keys())[site]
                                    })
                data_dirty['data'].append({"site" : list(sites.keys())[site], "item" : tag.text})


                with open(f"data_dirty_{pd.Timestamp('today'):%d-%m-%Y-%H-%M}.json", "w", encoding="utf-8") as file:
                      json.dump(data_dirty, file, ensure_ascii = False)
                        
            else:
                break                                
                
        if(exitFlag):
            df = pd.concat([pd.DataFrame(data['data']), df], ignore_index=True)
            df_dirty = pd.concat([pd.DataFrame(data_dirty['data']), df_dirty], ignore_index=True)        
            sites[list(sites.keys())[site]][1] = df.iloc[0]['title']   
          
    return df, df_dirty

In [9]:
df, df_dirty  = parsing(df, df_dirty) 
df.sample(8)

C:\Users\ASUS\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\ASUS\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constr

,title,category,pubdate,site
27,Путин назначил прокуроров четырех новых регион...,Политика,"Thu, 29 Dec 2022 16:45:00 +0300",ВЕДОМОСТИ
33,Путин подписал закон о штрафах до 5 млн рублей...,Общество,"Thu, 29 Dec 2022 15:48:42 +0300",ВЕДОМОСТИ
386,Россияне посетили пустыню в ОАЭ и разочаровали...,Путешествия,"Thu, 29 Dec 2022 20:21:00 +0300",LENTA.RU
65,В Совфеде раскритиковали идею конфисковывать и...,Политика,"Thu, 29 Dec 2022 10:45:02 +0300",ВЕДОМОСТИ
111,Устроивший стрельбу в пермском вузе приговорен...,Общество,"Wed, 28 Dec 2022 15:50:32 +0300",ВЕДОМОСТИ
447,Кадыров анонсировал более жестокую операцию «В...,Россия,"Thu, 29 Dec 2022 18:08:00 +0300",LENTA.RU
462,Хоркина высказалась о смене гражданства россий...,Спорт,"Thu, 29 Dec 2022 17:27:01 +0300",LENTA.RU
402,Банкам из недружественных стран разрешили выво...,Экономика,"Thu, 29 Dec 2022 19:26:00 +0300",LENTA.RU


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529 entries, 0 to 528
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     529 non-null    object
 1   category  529 non-null    object
 2   pubdate   529 non-null    object
 3   site      529 non-null    object
dtypes: object(4)
memory usage: 16.7+ KB


In [11]:
df_dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529 entries, 0 to 528
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   site    529 non-null    object
 1   item    529 non-null    object
dtypes: object(2)
memory usage: 8.4+ KB


In [12]:
df.head(5)

,title,category,pubdate,site
0,Путин подписал закон о создании единой базы ге...,Технологии,"Thu, 29 Dec 2022 22:41:42 +0300",ВЕДОМОСТИ
1,Умер трехкратный чемпион мира по футболу Пеле,Общество,"Thu, 29 Dec 2022 22:19:00 +0300",ВЕДОМОСТИ
2,Путин утвердил приостановку взыскания долгов с...,Общество,"Thu, 29 Dec 2022 21:57:07 +0300",ВЕДОМОСТИ
3,Киев заявил о готовности расследовать инцидент...,Политика,"Thu, 29 Dec 2022 21:32:44 +0300",ВЕДОМОСТИ
4,Путин подписал закон о предупреждении рисков п...,Бизнес,"Thu, 29 Dec 2022 21:12:16 +0300",ВЕДОМОСТИ


In [13]:
df = df.drop_duplicates()

In [14]:
# Создадим копию данных для возможного отката назад и сохраним их
df_init = df.copy(deep=True)

with open("df_init", "w", encoding="utf-8") as file:
                  json.dump(df_init.to_dict(), file, ensure_ascii = False)
        
with open("sites.json", "w", encoding="utf-8") as file:
                  json.dump(sites, file, ensure_ascii = False)           

In [15]:
df = pd.read_json("df_init")
df.head()

,title,category,pubdate,site
0,Путин подписал закон о создании единой базы ге...,Технологии,"Thu, 29 Dec 2022 22:41:42 +0300",ВЕДОМОСТИ
1,Умер трехкратный чемпион мира по футболу Пеле,Общество,"Thu, 29 Dec 2022 22:19:00 +0300",ВЕДОМОСТИ
2,Путин утвердил приостановку взыскания долгов с...,Общество,"Thu, 29 Dec 2022 21:57:07 +0300",ВЕДОМОСТИ
3,Киев заявил о готовности расследовать инцидент...,Политика,"Thu, 29 Dec 2022 21:32:44 +0300",ВЕДОМОСТИ
4,Путин подписал закон о предупреждении рисков п...,Бизнес,"Thu, 29 Dec 2022 21:12:16 +0300",ВЕДОМОСТИ


## Обаботка и подготовка данных

### Работа с различными категориями

Выделим основные категории, переименуем их. В случае появления новых категорий, будет отрпавлять в категорию "Всякое другое".

In [16]:
df['category'] = np.where(((df['category']=='Экономика') | (df['category']=='Бизнес') 
                           | (df['category']=='Финансы') | (df['category']=='Бизнес / Транспорт')),
                          'Экономика и бизнес', df['category']) 

df['category'] = np.where(((df['category']=='Моя страна') | (df['category']=='Москва')
                           | (df['category']=='Город') | (df['category']=='Доктор Питер')
                           | (df['category']=='Среда обитания') | (df['category']=='69-я параллель')
                           | (df['category']=='Северо-Запад')), 'Россия', df['category'])  

df['category'] = np.where(((df['category']=='Международная панорама') | (df['category']=='Бывший СССР')),
                          'Мир', df['category'])

df['category'] = np.where(((df['category']=='Силовые структуры') | (df['category']=='Космос')),
                          'Армия и ОПК', df['category']) 

df['category'] = np.where((df['category']=='Технологии'),'Наука и техника', df['category'])

df['category'] = np.where(((df['category']=='Власть')),'Политика', df['category']) 

df['category'] = np.where(((df['category']=='Из жизни') 
                           | (df['category']=='Биографии и справки') | (df['category']=='Особое мнение')
                          | (df['category']=='новости')), 'Происшествия', df['category'])

df['category'] = np.where((df['category']=='Туризм и отдых') | (df['category']=='Туризм'),
                          'Путешествия', df['category']) 

df['category'] = np.where((df['category']=='Афиша Plus'),'Культура', df['category'])

df['category'] = np.where((df['category']=='Забота о себе'),'Здоровье', df['category'])

df['category'] = np.where(((df['category']=='Ценности') | (df['category']=='Медиа')),'Интернет и СМИ', df['category']) 

df['category'] = np.where(((df['category']=='Авто') 
                           | (df['category']=='Недвижимость')),'Недвижимость и Авто', df['category'])

In [17]:
df['category'].value_counts()

Политика               87
Экономика и бизнес     87
Общество               84
Мир                    75
Россия                 44
Происшествия           29
Спорт                  28
Интернет и СМИ         24
Наука и техника        23
Армия и ОПК            20
Культура                9
Здоровье                9
Путешествия             8
Недвижимость и Авто     2
Name: category, dtype: int64

In [18]:
new_category = ['Политика',
 'Экономика и бизнес',
 'Мир',
 'Россия',
 'Происшествия',
 'Наука и техника',
 'Интернет и СМИ',
 'Армия и ОПК',
 'Спорт',
 'Путешествия',
 'Культура',
 'Здоровье'  ]

new_category

['Политика',
 'Экономика и бизнес',
 'Мир',
 'Россия',
 'Происшествия',
 'Наука и техника',
 'Интернет и СМИ',
 'Армия и ОПК',
 'Спорт',
 'Путешествия',
 'Культура',
 'Здоровье']

In [19]:
for cat in list(df['category'].value_counts().index):
     if cat not in new_category:
       df['category'] = np.where((df['category'] == cat),'Всякое другое', df['category'])

In [20]:
df['category'].value_counts()

Политика              87
Экономика и бизнес    87
Всякое другое         86
Мир                   75
Россия                44
Происшествия          29
Спорт                 28
Интернет и СМИ        24
Наука и техника       23
Армия и ОПК           20
Культура               9
Здоровье               9
Путешествия            8
Name: category, dtype: int64

### Создадим DataFrame с нумерованным списком категорий

In [21]:
df_category = pd.DataFrame(df['category'].unique())
df_category = df_category.rename_axis('id').reset_index()
df_category = df_category.rename(columns={0:'category'})
neworder = ['id','category'] 
df_category = df_category.reindex(columns=neworder)
df_category

,id,category
0,0,Наука и техника
1,1,Всякое другое
2,2,Политика
3,3,Экономика и бизнес
4,4,Интернет и СМИ
5,5,Мир
6,6,Россия
7,7,Происшествия
8,8,Армия и ОПК
9,9,Культура


### Создадим DataFrame с нумерованным списком сайтов-источников

In [22]:
df_sites  = pd.DataFrame(df['site'].unique())
df_sites  = df_sites.rename_axis('id').reset_index()
df_sites  = df_sites.rename(columns={0:'site'})
neworder = ['id','site']  
df_sites  = df_sites.reindex(columns=neworder)
df_sites 

,id,site
0,0,ВЕДОМОСТИ
1,1,TASS
2,2,LENTA.RU
3,3,Фонтонка.ру


### Создадим DataFrame с нумерованным списком новостей: id, номер категории, номер сайта и дата

In [23]:
df_news = df.copy()
df_news['date_of_week'] = 1
df_news['date_and_time'] = 1

Разделим даты на день недели и все остальное.

In [24]:
for index, row in enumerate(df_news['pubdate'].str.split(',')):
    #print(row)
    df_news['date_of_week'][index] = row[0]
    df_news['date_and_time'][index] = " ".join(row[1].split()[:-1])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21532\1006716051.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['date_of_week'][index] = row[0]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21532\1006716051.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['date_and_time'][index] = " ".join(row[1].split()[:-1])


Пронумеруем наши новости

In [25]:
df_news = df_news.drop(columns = ['pubdate'])
#df_news['id'] = df_news.index
df_news.sample(7)

,title,category,site,date_of_week,date_and_time
53,В Энгельсе сработала система ПВО,Политика,ВЕДОМОСТИ,Thu,29 Dec 2022 12:49:38
349,Директора российского центра творчества уволил...,Россия,LENTA.RU,Thu,29 Dec 2022 23:45:00
484,Дрейк ответил на обвинения девушки в абьюзе,Культура,LENTA.RU,Thu,29 Dec 2022 16:42:00
224,Мизулина назвала затраты организаторов информа...,Всякое другое,TASS,Fri,30 Dec 2022 03:20:16
205,В проливе Дарданеллы приостановили судоходство...,Мир,TASS,Fri,30 Dec 2022 06:37:15
473,Временный скачок цен на электроэнергию объяснили,Россия,LENTA.RU,Thu,29 Dec 2022 16:56:00
65,В Совфеде раскритиковали идею конфисковывать и...,Политика,ВЕДОМОСТИ,Thu,29 Dec 2022 10:45:02


Кодируем категории новостей и названия сайтов.

Проверим, что все корректно работает!

In [26]:
df_news.sample(5)

,title,category,site,date_of_week,date_and_time
33,Путин подписал закон о штрафах до 5 млн рублей...,Всякое другое,ВЕДОМОСТИ,Thu,29 Dec 2022 15:48:42
52,В Брянской области сообщили о сбитых беспилотн...,Политика,ВЕДОМОСТИ,Thu,29 Dec 2022 12:53:32
179,Путин поговорил по телефону с загадавшей побыв...,Всякое другое,ВЕДОМОСТИ,Tue,27 Dec 2022 16:14:18
21,Белгородский губернатор сообщил об обстреле Ше...,Всякое другое,ВЕДОМОСТИ,Thu,29 Dec 2022 17:39:04
472,В России снимут сериал об уникальной спасатель...,Россия,LENTA.RU,Thu,29 Dec 2022 16:59:00


In [27]:
df_news['category_num'] = 0
for ind, row in enumerate(df_news['category']):
    for i, cat in enumerate(df_category['category']):
        if row == cat:
            df_news['category_num'][ind] = i

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21532\2545184561.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['category_num'][ind] = i


In [28]:
df_news['site_num'] = 0
for ind, row in enumerate(df_news['site']):
    for i, cat in enumerate(df_sites['site']):
        if row == cat:
            df_news['site_num'][ind] = i

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21532\2594518947.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['site_num'][ind] = i


Все супер! Удалим избыточные столбцы 'site', 'category'.

In [29]:
df_news = df_news.drop(columns = ['site', 'category'])
neworder = ['title', 'category_num', 'site_num', 'date_and_time', 'date_of_week']  
df_news = df_news.reindex(columns=neworder)
df_news.sample(7)

,title,category_num,site_num,date_and_time,date_of_week
160,ЦБ предложил ввести потребительские кредиты дл...,3,0,27 Dec 2022 20:36:19,Tue
293,"В РФ вводится термин ""художник-ювелир"" и устан...",1,1,29 Dec 2022 22:52:52,Thu
288,Школьники из районов Тверской области смогут п...,1,1,29 Dec 2022 23:03:14,Thu
459,Россия запретит вывоз спирта за пределы страны...,3,2,29 Dec 2022 17:38:00,Thu
63,Kaspersky сообщил о новой схеме мошенничества ...,0,0,29 Dec 2022 11:15:36,Thu
32,Минобороны Белоруссии сообщило подробности пад...,2,0,29 Dec 2022 15:55:23,Thu
93,США и ЕС призвали к срочной деэскалации в Косове,2,0,28 Dec 2022 19:07:22,Wed


Кажется, что для наших витрин сами названия новостей не нужны. Тем более у нас есть сохраненная резервная копи данных. 
Удалим названия новостей. 

In [30]:
df_news = df_news.drop(columns = ['title'])
df_news.sample(7)

,category_num,site_num,date_and_time,date_of_week
132,3,0,28 Dec 2022 13:30:36,Wed
232,3,1,30 Dec 2022 02:45:57,Fri
353,10,2,29 Dec 2022 23:07:39,Thu
425,9,2,29 Dec 2022 18:43:00,Thu
473,6,2,29 Dec 2022 16:56:00,Thu
184,3,0,27 Dec 2022 15:24:37,Tue
286,5,1,29 Dec 2022 23:12:02,Thu


### Вывод

Теперь у нас есть три прекрасные нормализованные таблицы (пока еще DataFrame) для загрузки их в PostgreSQL.

In [31]:
df_news.sample(7)

,category_num,site_num,date_and_time,date_of_week
84,1,0,28 Dec 2022 21:20:14,Wed
118,3,0,28 Dec 2022 15:22:16,Wed
217,1,1,30 Dec 2022 05:17:28,Fri
120,2,0,28 Dec 2022 15:03:48,Wed
319,6,2,30 Dec 2022 04:25:35,Fri
310,5,2,30 Dec 2022 05:39:00,Fri
303,4,2,30 Dec 2022 06:26:00,Fri


## Заполнение таблиц БД PostgreSQL

Подключение к БД PostgreSQL

In [32]:
import psycopg2

conn = psycopg2.connect(
    database="employees",
    user="pdn",
    password="admin",
    host="host.docker.internal",
    port="25432"
)

cur = conn.cursor()

In [35]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS CATEGORIES_10 (
    ID INT PRIMARY KEY,
    category CHARACTER VARYING(100) NOT NULL 
)
"""

conn.autocommit = True
cur.execute(create_posts_table)

In [36]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS SITES_10 (
    ID INT PRIMARY KEY,
    SITE CHARACTER VARYING(80) NOT NULL 
)
"""

conn.autocommit = True
cur.execute(create_posts_table)

In [37]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS SS_NEWS_10 (
    ID INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    category_num INTEGER NOT NULL, 
    site_num INTEGER NOT NULL,
    date_and_time TIMESTAMP NOT NULL, 
    date_of_week CHARACTER VARYING(30) NOT NULL,
    CONSTRAINT fk_category_num
      FOREIGN KEY(category_num)
          REFERENCES CATEGORIES_10(ID)
          ON DELETE CASCADE,
    CONSTRAINT fk_site_num
      FOREIGN KEY(site_num)
          REFERENCES SITES_10(ID)
          ON DELETE CASCADE  
    )
"""

conn.autocommit = True
cur.execute(create_posts_table)

In [38]:
df_shema_category = list(df_category.itertuples(index=False, name=None))
df_shema_category

[(0, 'Наука и техника'),
 (1, 'Всякое другое'),
 (2, 'Политика'),
 (3, 'Экономика и бизнес'),
 (4, 'Интернет и СМИ'),
 (5, 'Мир'),
 (6, 'Россия'),
 (7, 'Происшествия'),
 (8, 'Армия и ОПК'),
 (9, 'Культура'),
 (10, 'Спорт'),
 (11, 'Здоровье'),
 (12, 'Путешествия')]

In [39]:
df_shema_site = list(df_sites.itertuples(index=False, name=None))
df_shema_site

[(0, 'ВЕДОМОСТИ'), (1, 'TASS'), (2, 'LENTA.RU'), (3, 'Фонтонка.ру')]

In [40]:
df_shema_news= list(df_news.itertuples(index=False, name=None))
df_shema_news[:10]

[(0, 0, '29 Dec 2022 22:41:42', 'Thu'),
 (1, 0, '29 Dec 2022 22:19:00', 'Thu'),
 (1, 0, '29 Dec 2022 21:57:07', 'Thu'),
 (2, 0, '29 Dec 2022 21:32:44', 'Thu'),
 (3, 0, '29 Dec 2022 21:12:16', 'Thu'),
 (2, 0, '29 Dec 2022 20:52:17', 'Thu'),
 (3, 0, '29 Dec 2022 20:44:34', 'Thu'),
 (2, 0, '29 Dec 2022 20:22:18', 'Thu'),
 (1, 0, '29 Dec 2022 19:58:09', 'Thu'),
 (2, 0, '29 Dec 2022 19:34:50', 'Thu')]

In [41]:
data_category = ", ".join(["%s"] * len(df_category))

insert_query = (
    f"INSERT INTO CATEGORIES_10 (ID, category) VALUES {data_category}"
)

conn.autocommit = True
cur = conn.cursor()
cur.execute(insert_query, df_shema_category)

In [42]:
data_site = ", ".join(["%s"] * len(df_sites))

insert_query = (
    f"INSERT INTO SITES_10 (ID, site) VALUES {data_site}"
)

conn.autocommit = True
cur = conn.cursor()
cur.execute(insert_query, df_shema_site)

In [43]:
data_news = ", ".join(["%s"] * len(df_news))

insert_query = (
    f"INSERT INTO SS_NEWS_10 (category_num, site_num, date_and_time, date_of_week) VALUES {data_news}"
)

conn.autocommit = True
cur = conn.cursor()
cur.execute(insert_query, df_shema_news)

In [44]:
#  Посмотрим на данные в таблице NEWS
cur.execute("SELECT * FROM SS_NEWS_10 LIMIT 10 ")
dirs = cur.fetchall()
for row in dirs[-5:]:
    
    print(row)

(6, 2, 0, datetime.datetime(2022, 12, 29, 20, 52, 17), 'Thu')
(7, 3, 0, datetime.datetime(2022, 12, 29, 20, 44, 34), 'Thu')
(8, 2, 0, datetime.datetime(2022, 12, 29, 20, 22, 18), 'Thu')
(9, 1, 0, datetime.datetime(2022, 12, 29, 19, 58, 9), 'Thu')
(10, 2, 0, datetime.datetime(2022, 12, 29, 19, 34, 50), 'Thu')


# Инкрементальный режим

In [45]:
site = pd.read_json(r"C:\Users\ASUS\OneDrive\Рабочий стол\AI\DE_1t\docker_3.1\sample project\Sample project\dags\scripts/sites.json")
site 

,Фонтонка.ру,LENTA.RU,TASS,ВЕДОМОСТИ
0,https://www.fontanka.ru/fontanka.rss,https://lenta.ru/rss,https://tass.ru/rss/v2.xml,https://www.vedomosti.ru/rss/news
1,"Мотя, Добрыня и канадские пумы сыграют в сказк...",Историк назвал основную причина распада СССР,Посольство РФ сообщило об отсутствии данных о ...,Путин подписал закон о создании единой базы ге...


In [46]:
sites = {"data" : [] 
               }
for ind, row in enumerate(site):
    sites['data'].append({row: [site[row][0], site[row][1]]})
sites = dict(list(sites['data'][0].items()) + list(sites['data'][1].items())+list(sites['data'][2].items()) + list(sites['data'][3].items()))    

In [47]:
sites

{'Фонтонка.ру': ['https://www.fontanka.ru/fontanka.rss',
  'Мотя, Добрыня и канадские пумы сыграют в сказке Пушкина на цирковой лад'],
 'LENTA.RU': ['https://lenta.ru/rss',
  'Историк назвал основную причина распада СССР'],
 'TASS': ['https://tass.ru/rss/v2.xml',
  'Посольство РФ сообщило об отсутствии данных о пострадавших при пожаре в Камбодже россиянах'],
 'ВЕДОМОСТИ': ['https://www.vedomosti.ru/rss/news',
  'Путин подписал закон о создании единой базы генетической информации']}

In [49]:
#sites = sites.to_dict('list')
#sites 

In [50]:
# создадим пустые DataFrames, куда будем записывать данные
data = {"data" : []  }
data_dirty = {"data" : []  }
#df = pd.DataFrame(data['data'])
df_dirty = pd.DataFrame(data_dirty['data'])

In [51]:
df, df_dirty  = parsing(df, df_dirty) 
df.sample(8)

C:\Users\ASUS\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


,title,category,pubdate,site
228,В Тамбовской области объявили экстренное преду...,Всякое другое,"Fri, 30 Dec 2022 03:07:00 +0300",TASS
193,Суд приговорил военного к году и восьми месяца...,Всякое другое,"Tue, 27 Dec 2022 14:14:21 +0300",ВЕДОМОСТИ
164,Суд на Украине признал конституционным закон о...,Всякое другое,"Tue, 27 Dec 2022 18:46:23 +0300",ВЕДОМОСТИ
79,Правительство обновило состав комиссии по разв...,Политика,"Wed, 28 Dec 2022 22:48:40 +0300",ВЕДОМОСТИ
160,ЦБ предложил ввести потребительские кредиты дл...,Экономика и бизнес,"Tue, 27 Dec 2022 20:36:19 +0300",ВЕДОМОСТИ
31,Вучич отменил режим повышенной боевой готовнос...,Политика,"Thu, 29 Dec 2022 15:56:35 +0300",ВЕДОМОСТИ
357,Молдавия заявила о прекращении отношений с рук...,Мир,"Thu, 29 Dec 2022 22:33:00 +0300",LENTA.RU
93,США и ЕС призвали к срочной деэскалации в Косове,Политика,"Wed, 28 Dec 2022 19:07:22 +0300",ВЕДОМОСТИ


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 529 entries, 0 to 528
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     529 non-null    object
 1   category  529 non-null    object
 2   pubdate   529 non-null    object
 3   site      529 non-null    object
dtypes: object(4)
memory usage: 36.8+ KB


In [53]:
df_dirty.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

In [54]:
df.head(5)

,title,category,pubdate,site
0,Путин подписал закон о создании единой базы ге...,Наука и техника,"Thu, 29 Dec 2022 22:41:42 +0300",ВЕДОМОСТИ
1,Умер трехкратный чемпион мира по футболу Пеле,Всякое другое,"Thu, 29 Dec 2022 22:19:00 +0300",ВЕДОМОСТИ
2,Путин утвердил приостановку взыскания долгов с...,Всякое другое,"Thu, 29 Dec 2022 21:57:07 +0300",ВЕДОМОСТИ
3,Киев заявил о готовности расследовать инцидент...,Политика,"Thu, 29 Dec 2022 21:32:44 +0300",ВЕДОМОСТИ
4,Путин подписал закон о предупреждении рисков п...,Экономика и бизнес,"Thu, 29 Dec 2022 21:12:16 +0300",ВЕДОМОСТИ


In [55]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 529 entries, 0 to 528
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     529 non-null    object
 1   category  529 non-null    object
 2   pubdate   529 non-null    object
 3   site      529 non-null    object
dtypes: object(4)
memory usage: 20.7+ KB


In [56]:
# Создадим копию данных для возможного отката назад и сохраним их
df_init = df.copy(deep=True)

with open("df_init", "w", encoding="utf-8") as file:
                  json.dump(df_init.to_dict(), file, ensure_ascii = False)
        
with open("sites.json", "w", encoding="utf-8") as file:
                  json.dump(sites, file, ensure_ascii = False)           

## Обаботка и подготовка данных

### Работа с различными категориями

Выделим основные категории, переименуем их. В случае появления новых категорий, будет отрпавлять в категорию "Всякое другое".

In [57]:
df['category'] = np.where(((df['category']=='Экономика') | (df['category']=='Бизнес') 
                           | (df['category']=='Финансы') | (df['category']=='Бизнес / Транспорт')),
                          'Экономика и бизнес', df['category']) 

df['category'] = np.where(((df['category']=='Моя страна') | (df['category']=='Москва')
                           | (df['category']=='Город') | (df['category']=='Доктор Питер')
                           | (df['category']=='Среда обитания')),'Россия', df['category'])

df['category'] = np.where(((df['category']=='Международная панорама') | (df['category']=='Бывший СССР')),
                          'Мир', df['category'])

df['category'] = np.where(((df['category']=='Силовые структуры') | (df['category']=='Космос')),
                          'Армия и ОПК', df['category']) 

df['category'] = np.where((df['category']=='Технологии'),'Наука и техника', df['category'])

df['category'] = np.where(((df['category']=='Власть')),'Политика', df['category']) 

df['category'] = np.where(((df['category']=='Из жизни') 
                           | (df['category']=='Биографии и справки') | (df['category']=='Особое мнение')),
                          'Происшествия', df['category'])

df['category'] = np.where((df['category']=='Туризм и отдых') | (df['category']=='Туризм'),
                          'Путешествия', df['category']) 

df['category'] = np.where((df['category']=='Афиша Plus'),'Культура', df['category'])

df['category'] = np.where((df['category']=='Забота о себе'),'Здоровье', df['category'])

df['category'] = np.where((df['category']=='Ценности'),'Интернет и СМИ', df['category'])

df['category'] = np.where(((df['category']=='Авто') 
                           | (df['category']=='Недвижимость')),'Недвижимость и Авто', df['category'])

In [58]:
df['category'].value_counts()

Политика              87
Экономика и бизнес    87
Всякое другое         86
Мир                   75
Россия                44
Происшествия          29
Спорт                 28
Интернет и СМИ        24
Наука и техника       23
Армия и ОПК           20
Культура               9
Здоровье               9
Путешествия            8
Name: category, dtype: int64

In [59]:
new_category = ['Политика',
 'Экономика и бизнес',
 'Мир',
 'Россия',
 'Происшествия',
 'Наука и техника',
 'Интернет и СМИ',
 'Армия и ОПК',
 'Спорт',
 'Путешествия',
 'Культура',
 'Здоровье'  ]

new_category

['Политика',
 'Экономика и бизнес',
 'Мир',
 'Россия',
 'Происшествия',
 'Наука и техника',
 'Интернет и СМИ',
 'Армия и ОПК',
 'Спорт',
 'Путешествия',
 'Культура',
 'Здоровье']

In [60]:
for cat in list(df['category'].value_counts().index):
     if cat not in new_category:
       df['category'] = np.where((df['category'] == cat),'Всякое другое', df['category'])

In [61]:
df['category'].value_counts()

Политика              87
Экономика и бизнес    87
Всякое другое         86
Мир                   75
Россия                44
Происшествия          29
Спорт                 28
Интернет и СМИ        24
Наука и техника       23
Армия и ОПК           20
Культура               9
Здоровье               9
Путешествия            8
Name: category, dtype: int64

### Создадим DataFrame с нумерованным списком категорий

In [62]:
df_category = pd.DataFrame(df['category'].unique())
df_category = df_category.rename_axis('id').reset_index()
df_category = df_category.rename(columns={0:'category'})
neworder = ['id','category'] 
df_category = df_category.reindex(columns=neworder)
df_category

,id,category
0,0,Наука и техника
1,1,Всякое другое
2,2,Политика
3,3,Экономика и бизнес
4,4,Интернет и СМИ
5,5,Мир
6,6,Россия
7,7,Происшествия
8,8,Армия и ОПК
9,9,Культура


### Создадим DataFrame с нумерованным списком сайтов-источников

In [63]:
df_sites  = pd.DataFrame(df['site'].unique())
df_sites  = df_sites.rename_axis('id').reset_index()
df_sites  = df_sites.rename(columns={0:'site'})
neworder = ['id','site']  
df_sites  = df_sites.reindex(columns=neworder)
df_sites 

,id,site
0,0,ВЕДОМОСТИ
1,1,TASS
2,2,LENTA.RU
3,3,Фонтонка.ру


### Создадим DataFrame с нумерованным списком новостей: id, номер категории, номер сайта и дата

In [64]:
df_news = df.copy()
df_news['date_of_week'] = 1
df_news['date_and_time'] = 1

Разделим даты на день недели и все остальное.

In [65]:
for index, row in enumerate(df_news['pubdate'].str.split(',')):
    #print(row)
    df_news['date_of_week'][index] = row[0]
    df_news['date_and_time'][index] = " ".join(row[1].split()[:-1])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21532\1006716051.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['date_of_week'][index] = row[0]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21532\1006716051.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['date_and_time'][index] = " ".join(row[1].split()[:-1])


In [66]:
df_news = df_news.drop(columns = ['pubdate'])
df_news.sample(7)

,title,category,site,date_of_week,date_and_time
189,СК возбудил уголовное дело после убийства семь...,Всякое другое,ВЕДОМОСТИ,Tue,27 Dec 2022 14:31:22
219,В ЛНР сообщили об украинском обстреле Брянки и...,Происшествия,TASS,Fri,30 Dec 2022 05:08:55
83,В Севастополе опровергли введение комендантско...,Всякое другое,ВЕДОМОСТИ,Wed,28 Dec 2022 21:42:37
471,В доставленных на Землю лунных породах нашли э...,Наука и техника,LENTA.RU,Thu,29 Dec 2022 16:59:00
440,МИД Белоруссии выразил протест украинскому пос...,Мир,LENTA.RU,Thu,29 Dec 2022 18:16:00
144,На Урале мужчина получил пожизненный срок за у...,Всякое другое,ВЕДОМОСТИ,Wed,28 Dec 2022 11:34:55
42,Латвия не будет аннулировать рабочие визы сотр...,Интернет и СМИ,ВЕДОМОСТИ,Thu,29 Dec 2022 14:45:42


Проверим, что все корректно работает!

In [67]:
df_news.sample(5)

,title,category,site,date_of_week,date_and_time
501,"Дух реки и нарядные ели. Посмотрите, как Леноб...",Всякое другое,Фонтонка.ру,Fri,30 Dec 2022 00:59:48
224,Мизулина назвала затраты организаторов информа...,Всякое другое,TASS,Fri,30 Dec 2022 03:20:16
470,Путин подписал закон о запрете принудительного...,Россия,LENTA.RU,Thu,29 Dec 2022 17:00:00
486,Названа стоимость «крабового сафари» на Крайне...,Россия,LENTA.RU,Thu,29 Dec 2022 16:41:00
474,Больница по ошибке сообщила пациентам о раке в...,Происшествия,LENTA.RU,Thu,29 Dec 2022 16:56:00


In [68]:
df_news['category_num'] = 0
for ind, row in enumerate(df_news['category']):
    for i, cat in enumerate(df_category['category']):
        if row == cat:
            df_news['category_num'][ind] = i

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21532\2545184561.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['category_num'][ind] = i


In [69]:
df_news['site_num'] = 0
for ind, row in enumerate(df_news['site']):
    for i, cat in enumerate(df_sites['site']):
        if row == cat:
            df_news['site_num'][ind] = i

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21532\2594518947.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['site_num'][ind] = i


Все супер! Удалим избыточные столбцы 'site', 'category'.

In [70]:
df_news = df_news.drop(columns = ['site', 'category'])
neworder = ['title', 'category_num', 'site_num', 'date_and_time', 'date_of_week']  
df_news = df_news.reindex(columns=neworder)
df_news.sample(7)

,title,category_num,site_num,date_and_time,date_of_week
2,Путин утвердил приостановку взыскания долгов с...,1,0,29 Dec 2022 21:57:07,Thu
416,Шансы Италии стать гарантом мирного договора п...,5,2,29 Dec 2022 19:02:00,Thu
401,Пассажиров российского самолета эвакуировали и...,12,2,29 Dec 2022 19:29:00,Thu
186,В Минсельхозе оценили ситуацию на рынке сахара,3,0,27 Dec 2022 15:01:40,Tue
163,Курс доллара превысил 71 рубль,3,0,27 Dec 2022 19:11:14,Tue
474,Больница по ошибке сообщила пациентам о раке в...,7,2,29 Dec 2022 16:56:00,Thu
324,России предсказали рекордный профицит внешней ...,3,2,30 Dec 2022 03:17:00,Fri


Кажется, что для наших витрин сами названия новостей не нужны. Тем более у нас есть сохраненная резервная копи данных. 
Удалим названия новостей. 

In [71]:
df_news = df_news.drop(columns = ['title'])
df_news.sample(7)

,category_num,site_num,date_and_time,date_of_week
94,2,0,28 Dec 2022 19:02:27,Wed
385,5,2,29 Dec 2022 20:22:00,Thu
76,2,0,29 Dec 2022 07:55:16,Thu
265,1,1,30 Dec 2022 00:08:44,Fri
458,10,2,29 Dec 2022 17:41:35,Thu
309,6,2,30 Dec 2022 05:46:30,Fri
383,8,2,29 Dec 2022 20:27:00,Thu


### Вывод

Теперь у нас есть три прекрасные нормализованные таблицы (пока еще DataFrame) для загрузки их в PostgreSQL.

In [72]:
df_news.sample(7)

,category_num,site_num,date_and_time,date_of_week
113,1,0,28 Dec 2022 15:46:15,Wed
293,1,1,29 Dec 2022 22:52:52,Thu
468,9,2,29 Dec 2022 17:06:00,Thu
248,9,1,30 Dec 2022 01:02:58,Fri
287,10,1,29 Dec 2022 23:06:19,Thu
436,6,2,29 Dec 2022 18:24:00,Thu
225,5,1,30 Dec 2022 03:19:14,Fri


## Заполнение таблиц БД PostgreSQL

Подключение к БД PostgreSQL

In [73]:
import psycopg2

conn = psycopg2.connect(
    database="airflow",
    user="airflow",
    password="airflow",
    host="host.docker.internal",
    port="35432"
)

cur = conn.cursor()

In [74]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS SS_NEWS_10 (
    ID INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    category_num INTEGER NOT NULL, 
    site_num INTEGER NOT NULL,
    date_and_time TIMESTAMP NOT NULL, 
    date_of_week CHARACTER VARYING(30) NOT NULL,
    CONSTRAINT fk_category_num
      FOREIGN KEY(category_num)
          REFERENCES CATEGORY(ID)
          ON DELETE CASCADE,
    CONSTRAINT fk_site_num
      FOREIGN KEY(site_num)
          REFERENCES SITE(ID)
          ON DELETE CASCADE  
    )
"""

conn.autocommit = True
cur.execute(create_posts_table)

In [75]:
df_shema_news= list(df_news.itertuples(index=False, name=None))
df_shema_news[:10]

[(0, 0, '29 Dec 2022 22:41:42', 'Thu'),
 (1, 0, '29 Dec 2022 22:19:00', 'Thu'),
 (1, 0, '29 Dec 2022 21:57:07', 'Thu'),
 (2, 0, '29 Dec 2022 21:32:44', 'Thu'),
 (3, 0, '29 Dec 2022 21:12:16', 'Thu'),
 (2, 0, '29 Dec 2022 20:52:17', 'Thu'),
 (3, 0, '29 Dec 2022 20:44:34', 'Thu'),
 (2, 0, '29 Dec 2022 20:22:18', 'Thu'),
 (1, 0, '29 Dec 2022 19:58:09', 'Thu'),
 (2, 0, '29 Dec 2022 19:34:50', 'Thu')]

In [76]:
data_news = ", ".join(["%s"] * len(df_news))

insert_query = (
    f"INSERT INTO SS_NEWS (category_num, site_num, date_and_time, date_of_week) VALUES {data_news}"
)

conn.autocommit = True
cur = conn.cursor()
cur.execute(insert_query, df_shema_news)

In [77]:
#  Посмотрим на данные в таблице NEWS
cur.execute("SELECT * FROM SS_NEWS_10 LIMIT 10 ")
dirs = cur.fetchall()
for row in dirs[-5:]:
    
    print(row)

(6, 2, 0, datetime.datetime(2022, 12, 29, 20, 52, 17), 'Thu')
(7, 3, 0, datetime.datetime(2022, 12, 29, 20, 44, 34), 'Thu')
(8, 2, 0, datetime.datetime(2022, 12, 29, 20, 22, 18), 'Thu')
(9, 1, 0, datetime.datetime(2022, 12, 29, 19, 58, 9), 'Thu')
(10, 2, 0, datetime.datetime(2022, 12, 29, 19, 34, 50), 'Thu')


# Витрины

Создадим витрины с помощью SQL скриптов.

## Витрина 1

In [81]:
#Общее количество новостей из всех источников по данной категории за все время
#Общее количество новостей из всех источников по данной категории за последние сутки
#Среднее количество публикаций по данной категории за все время
#Среднее количество публикаций по данной категории за последние сутки

cur.execute("with nnum_4 as ( \
                    with num_4 as (select category_num, \
                                        round(avg(id)) as avg_per_last_day \
                            from ss_news \
                            where (cast(CURRENT_DATE as DATE)- cast(date_and_time as DATE)) < 1 \
                            group by category_num \
                            order by category_num ) \
                    select id, category, avg_per_last_day \
                    from categories ac \
                    join num_4 on num_4.category_num = ac.id \
                    ), \
                    nnum_1 as( \
                    with num_1 as (select category_num, \
                            sum(id) as sum_all \
                    from ss_news \
                    group by category_num \
                    order by category_num )  \
                    select id, category, sum_all \
                    from categories ac  \
                    join num_1 on num_1.category_num = ac.id \
                    ), \
                    nnum_2 as ( \
                    with num_2 as (select category_num, \
                                        sum(id) as sum_last_day \
                                 from ss_news \
                                 where (cast(CURRENT_DATE as DATE)- cast(date_and_time as DATE)) < 2  \
                                 group by category_num)  \
                    select ac.id, \
                           category, \
                           sum_last_day \
                    from categories ac  \
                    join num_2 on num_2.category_num = ac.id \
                    ), \
                    nnum_3 as ( \
                    with num_3 as (select category_num, \
                                        round(avg(id)) as avg_all \
                            from ss_news \
                            group by category_num \
                            order by category_num ) \
                    select id, category, avg_all \
                    from categories ac  \
                    join num_3 on num_3.category_num = ac.id \
                    ) \
                    select nnum_2.id, \
                           nnum_2.category, \
                           nnum_2.sum_last_day, \
                           nnum_1.sum_all, \
                           nnum_3.avg_all, \
                           nnum_4.avg_per_last_day \
                    from nnum_2 join nnum_1 \
                    on nnum_1.id = nnum_2.id \
                    join nnum_3  \
                    on nnum_1.id = nnum_3.id \
                    join nnum_4 \
                    on nnum_1.id = nnum_4.id")
dirs = cur.fetchall()
for row in dirs[-20:]:
    
    print(row)

(0, 'Наука и техника', 1515119612, 2337259414, Decimal('154010'), Decimal('192603'))
(1, 'Всякое другое', 2933457143, 4621189692, Decimal('116759'), Decimal('132801'))
(2, 'Политика', 1895857486, 3787721436, Decimal('107658'), Decimal('152861'))
(3, 'Экономика и бизнес', 2616364201, 4622416592, Decimal('116716'), Decimal('109282'))
(4, 'Интернет и СМИ', 2615022404, 2615022404, Decimal('129233'), Decimal('143053'))
(5, 'Мир', 3062508539, 3062508539, Decimal('109610'), Decimal('127997'))
(6, 'Армия и ОПК', 1577424264, 1577424264, Decimal('134466'), Decimal('133193'))
(7, 'Происшествия', 934445841, 934445841, Decimal('142816'), Decimal('157416'))
(8, 'Культура', 953079054, 953079054, Decimal('96534'), Decimal('102863'))
(9, 'Спорт', 1381171748, 1381171748, Decimal('115666'), Decimal('152891'))
(10, 'Россия', 986467063, 986467063, Decimal('96817'), Decimal('129930'))
(11, 'Здоровье', 702952534, 702952534, Decimal('134769'), Decimal('177466'))
(12, 'Путешествия', 577427374, 577427374, Decim

## Витрина 2

In [90]:
#Количество новостей данной категории для каждого из источников за последние сутки
#Количество новостей данной категории для каждого из источников за все время

cur.execute("with nn_1 as ( \
                with n_1 as (select category_num, site_num, \
                            sum(id) as sum_per_site \
                            from ss_news \
                               group by category_num, site_num \
                               order by category_num, site_num), \
                     s as (select id,  \
                                  site \
                           from sites) \
                select c.id,  \
                       c.category,  \
                       s.site, \
                       sum(n_1.sum_per_site) as sum_per_site_all \
                from categories as c \
                join n_1 on n_1.category_num = c.id \
                join s on n_1.site_num = s.id \
                group by c.id, s.site  \
                order by id  \
                ), \
                nn_2 as  \
                (with n_2 as (select category_num,  \
                                  site_num, \
                                  sum(id) as sum_per_site_last_day \
                             from SS_news \
                             where (cast(CURRENT_DATE as DATE)- cast(date_and_time as DATE)) < 2  \
                             group by category_num, site_num  \
                             order by category_num, site_num), \
                         s as (select id,  \
                                  site \
                           from aa_site) \
                select c.id,  \
                       c.category,  \
                       s.site, \
                       sum(n_2.sum_per_site_last_day) as sum_per_site_last_day \
                from categories as c \
                join n_2 on n_2.category_num = c.id \
                join s on n_2.site_num = s.id \
                group by c.id, s.site  \
                order by id, site \
                ) \
                select nn_1.id,  \
                       nn_1.category,  \
                       nn_1.site, \
                       nn_1.sum_per_site_all, \
                       nn_2.sum_per_site_last_day \
                from nn_1 join nn_2 \
                on nn_1.id = nn_2.id") 
dirs = cur.fetchall()
for row in dirs[-20:]:
    
    print(row)

(9, 'Спорт', 'TASS', Decimal('111612257'), Decimal('1117608950'))
(9, 'Спорт', 'TASS', Decimal('111612257'), Decimal('111612257'))
(9, 'Спорт', 'TASS', Decimal('111612257'), Decimal('238164623'))
(9, 'Спорт', 'Фонтонка.ру', Decimal('238164623'), Decimal('1117608950'))
(9, 'Спорт', 'Фонтонка.ру', Decimal('238164623'), Decimal('111612257'))
(9, 'Спорт', 'Фонтонка.ру', Decimal('238164623'), Decimal('238164623'))
(10, 'Россия', 'LENTA.RU', Decimal('997869385'), Decimal('997869385'))
(10, 'Россия', 'LENTA.RU', Decimal('997869385'), Decimal('1068175'))
(10, 'Россия', 'LENTA.RU', Decimal('997869385'), Decimal('43776618'))
(10, 'Россия', 'TASS', Decimal('1068175'), Decimal('997869385'))
(10, 'Россия', 'TASS', Decimal('1068175'), Decimal('1068175'))
(10, 'Россия', 'TASS', Decimal('1068175'), Decimal('43776618'))
(10, 'Россия', 'Фонтонка.ру', Decimal('43776618'), Decimal('997869385'))
(10, 'Россия', 'Фонтонка.ру', Decimal('43776618'), Decimal('1068175'))
(10, 'Россия', 'Фонтонка.ру', Decimal('43

## Витрина 3

In [94]:
#День, в который было сделано максимальное количество публикаций по данной категории
cur.execute(" with s as ( \
            select category_num, \
                        (first_value(cast(date_and_time as DATE)) OVER (PARTITION BY category_num)) as max_day \
                         from ss_news \
                         group by category_num, cast(date_and_time as DATE) \
                         order by category_num ) \
            select category_num, \
                   max(max_day) as day_with_max_news \
            from s  \
            group by category_num") 
dirs = cur.fetchall()
for row in dirs[-20:]:
    
    print(row)

(0, datetime.date(2022, 12, 27))
(1, datetime.date(2022, 12, 27))
(2, datetime.date(2022, 12, 27))
(3, datetime.date(2022, 12, 27))
(4, datetime.date(2022, 12, 29))
(5, datetime.date(2022, 12, 29))
(6, datetime.date(2022, 12, 29))
(7, datetime.date(2022, 12, 29))
(8, datetime.date(2022, 12, 29))
(9, datetime.date(2022, 12, 29))
(10, datetime.date(2022, 12, 29))
(11, datetime.date(2022, 12, 29))
(12, datetime.date(2022, 12, 29))


## Витрина 4

In [80]:
#Количество публикаций новостей данной категории по дням недели
cur.execute("with num as( select category_num, date_of_week, sum(id) as sum_pub \
                from ss_news \
                group by category_num, date_of_week \
                order by category_num, date_of_week \
                ) \
                select ac.id, \
                       category, \
                       date_of_week, \
                       sum_pub \
                from categories as ac \
                join num on num.category_num = ac.id")
dirs = cur.fetchall()
for row in dirs[-20:]:
    
    print(row)

(3, 'Экономика и бизнес', 'Thu', 1932659491)
(3, 'Экономика и бизнес', 'Fri', 616904811)
(4, 'Интернет и СМИ', 'Thu', 1617553997)
(4, 'Интернет и СМИ', 'Fri', 968155096)
(5, 'Мир', 'Thu', 2071409284)
(5, 'Мир', 'Fri', 908376216)
(6, 'Армия и ОПК', 'Thu', 837495330)
(6, 'Армия и ОПК', 'Fri', 617740125)
(7, 'Происшествия', 'Thu', 479499586)
(7, 'Происшествия', 'Fri', 383089966)
(8, 'Культура', 'Thu', 611839522)
(8, 'Культура', 'Fri', 272662490)
(9, 'Спорт', 'Thu', 971741601)
(9, 'Спорт', 'Fri', 360768179)
(10, 'Россия', 'Thu', 749907472)
(10, 'Россия', 'Fri', 206171558)
(11, 'Здоровье', 'Thu', 501787394)
(11, 'Здоровье', 'Fri', 179784703)
(12, 'Путешествия', 'Thu', 434031435)
(12, 'Путешествия', 'Fri', 109592874)
